POC DAX M30 : POC to test different scores with DAX30 data using both 30 min timeframe and 1 day timeframe.
This notebook is the second part : try differents algo : classification with random forest and regression with random forest and XGBoost

In [ ]:
from importlib import reload
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
import ta
import time
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,GridSearchCV
from sklearn import metrics
import pickle
import joblib
import xgboost as xgb


In [ ]:
PATH_DATA = "../../../Data/"
PATH_DATA_Quant = "../../../Data/dl_quant/"
PATH_DATA_Yahoo = "../../../Data/dl_yahoo/"

In [ ]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
df_dax_m30_d1 = pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA_clean.zip", sep=",",
                            index_col="Datetime", parse_dates=True, keep_default_na=True, low_memory=False)
df_dax_m30_d1.sort_index(inplace = True)
df_dax_m30_d1.info()

In [ ]:
df_model = df_dax_m30_d1[['m30_atr_atr14_norm', 'm30_pos_sma20_norm', 'm30_pos_hband_norm', 'm30_pos_lband_norm', 'm30_market_open', 'm30_return', 'm30_rsi14', 'm30_is_up', 'm30_pos_highest20',
                          'm30_hour_norm', 'd1_pos_sma20_norm', 'd1_pos_sma50_norm', 'd1_pos_hband_norm', 'd1_pos_lband_norm', 'd1_vol_vol20_norm', 'd1_BBW_BBw20', 'd1_pos_sma200', 'd1_pos_highest20', 'd1_rsi14', 'd1_return', 'm30_top_tp_touch']]
df_model.dropna(inplace=True)
df_model.describe()


In [ ]:
X_cols=df_model[['m30_atr_atr14_norm', 'm30_pos_sma20_norm', 'm30_pos_hband_norm', 'm30_pos_lband_norm', 'm30_market_open', 'm30_return', 'm30_rsi14', 'm30_is_up', 'm30_pos_highest20',
                          'm30_hour_norm', 'd1_pos_sma20_norm', 'd1_pos_sma50_norm', 'd1_pos_hband_norm', 'd1_pos_lband_norm', 'd1_vol_vol20_norm', 'd1_BBW_BBw20', 'd1_pos_sma200', 'd1_pos_highest20', 'd1_rsi14', 'd1_return']]
y_cols=df_model['m30_top_tp_touch']

In [ ]:
X_cols_valid=X_cols['2020-07-01':]
y_cols_valid=y_cols['2020-07-01':]
X_cols_train_test=X_cols[:'2020-07-01']
y_cols_train_test=y_cols[:'2020-07-01']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_cols_train_test, y_cols_train_test, random_state=0, train_size=0.7)


In [ ]:
y_train.describe()

In [ ]:
rand_frst_clf=RandomForestClassifier(n_estimators=100,oob_score=True,criterion='gini',random_state=28,class_weight="balanced")
rand_frst_clf.fit(X_train,y_train)

In [ ]:
y_pred_test=rand_frst_clf.predict(X_test)

In [ ]:
print(f"Accuracy :{accuracy_score(y_test,rand_frst_clf.predict(X_test),normalize=True)*100}")

In [ ]:
report=classification_report(y_true=y_test,y_pred=y_pred_test,target_names=['bad','TP touched'],output_dict=True)
df_report=pd.DataFrame(report).transpose()
df_report

In [ ]:
feat_imp=rand_frst_clf.feature_importances_
std_feat_imp = np.std([tree.feature_importances_ for tree in rand_frst_clf.estimators_], axis=0)
pd.Series(feat_imp,index=X_cols.columns).sort_values(ascending=False)

In [ ]:

forest_importances = pd.Series(feat_imp, index=X_cols.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std_feat_imp, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
# X_train_trim, , y_train, y_test = train_test_split(
#    X_cols_train_test, y_cols_train_test, random_state=0, train_size=0.7)
X_test_trim = pd.concat([X_cols_train_test['2015-04-01':'2015-06-30'],
                        X_cols_train_test['2015-10-01':'2015-12-31'], X_cols_train_test['2016-04-01':'2016-12-31'],X_cols_train_test['2018-01-01':'2018-03-31'],X_cols_train_test['2019-01-01':'2019-03-31']])
y_test_trim = pd.concat([y_cols_train_test['2015-04-01':'2015-06-30'],
                        y_cols_train_test['2015-10-01':'2015-12-31'], y_cols_train_test['2016-04-01':'2016-12-31'],y_cols_train_test['2018-01-01':'2018-03-31'],y_cols_train_test['2019-01-01':'2019-03-31']])

y_test_trim.describe()

In [ ]:
X_train_trim=X_cols_train_test.merge(X_test_trim,indicator=True,how='left').loc[lambda x : x['_merge']!='both']
X_train_trim.drop(axis=1, columns='_merge',inplace=True)
X_train_trim.describe()

In [ ]:
y_train_trim=y_cols_train_test[~y_cols_train_test.index.isin(y_test_trim.index)]
y_train_trim.describe()

In [ ]:
rand_frst_clf_trim=RandomForestClassifier(n_estimators=100,oob_score=True,criterion='gini',random_state=28,class_weight="balanced")
rand_frst_clf_trim.fit(X_train_trim,y_train_trim)

In [ ]:
y_pred_test_trim=rand_frst_clf_trim.predict(X_test_trim)
print(f"Accuracy :{accuracy_score(y_test_trim,rand_frst_clf_trim.predict(X_test_trim),normalize=True)*100}")

In [ ]:
report_trim=classification_report(y_true=y_test_trim,y_pred=y_pred_test_trim,target_names=['bad','TP touched'],output_dict=True)
df_report_trim=pd.DataFrame(report_trim).transpose()
df_report_trim

In [ ]:
feat_imp_trim=rand_frst_clf_trim.feature_importances_
std_feat_imp_trim = np.std([tree.feature_importances_ for tree in rand_frst_clf_trim.estimators_], axis=0)
pd.Series(feat_imp_trim,index=X_cols.columns).sort_values(ascending=False)

In [ ]:

forest_importances_trim = pd.Series(feat_imp_trim, index=X_cols.columns)

fig, ax = plt.subplots()
forest_importances_trim.plot.bar(yerr=std_feat_imp_trim, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
y_pred_valid=rand_frst_clf.predict(X_cols_valid)
y_pred_valid_trim=rand_frst_clf_trim.predict(X_cols_valid)
print(f"Accuracy shuffle:{metrics.accuracy_score(y_cols_valid,rand_frst_clf.predict(X_cols_valid),normalize=True)*100}")
print(f"Accuracy trim :{metrics.accuracy_score(y_cols_valid,rand_frst_clf_trim.predict(X_cols_valid),normalize=True)*100}")

In [ ]:
report_valid=metrics.classification_report(y_true=y_cols_valid,y_pred=y_pred_valid,target_names=['bad','TP touched'],output_dict=True)
df_report_valid=pd.DataFrame(report_valid).transpose()
report_valid_trim=metrics.classification_report(y_true=y_cols_valid,y_pred=y_pred_valid_trim,target_names=['bad','TP touched'],output_dict=True)
df_report_valid_trim=pd.DataFrame(report_valid_trim).transpose()
print(df_report_valid)
print(df_report_valid_trim)

In [ ]:
df_dax_m30_d1 = pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA_clean.zip", sep=",",
                            index_col="Datetime", parse_dates=True, keep_default_na=True, low_memory=False)
df_dax_m30_d1.sort_index(inplace = True)
df_model = df_dax_m30_d1[['m30_atr_atr14_norm', 'm30_pos_sma20_norm', 'm30_pos_hband_norm', 'm30_pos_lband_norm', 'm30_market_open', 'm30_return', 'm30_rsi14', 'm30_is_up', 'm30_pos_highest20',
                          'm30_hour_norm', 'd1_pos_sma20_norm', 'd1_pos_sma50_norm', 'd1_pos_hband_norm', 'd1_pos_lband_norm', 'd1_vol_vol20_norm', 'd1_BBW_BBw20', 'd1_pos_sma200', 'd1_pos_highest20', 'd1_rsi14', 'd1_return', 'm30_top_tp_touch']]
df_model.dropna(inplace=True)
X_cols=df_model[['m30_atr_atr14_norm', 'm30_pos_sma20_norm', 'm30_pos_hband_norm', 'm30_pos_lband_norm', 'm30_market_open', 'm30_return', 'm30_rsi14', 'm30_is_up', 'm30_pos_highest20',
                          'm30_hour_norm', 'd1_pos_sma20_norm', 'd1_pos_sma50_norm', 'd1_pos_hband_norm', 'd1_pos_lband_norm', 'd1_vol_vol20_norm', 'd1_BBW_BBw20', 'd1_pos_sma200', 'd1_pos_highest20', 'd1_rsi14', 'd1_return']]
y_cols=df_model['m30_top_tp_touch']
rand_frst_clf=joblib.load(PATH_DATA_Quant+"poc_dax_m30_rand_forest.pkl")

In [ ]:
df_model.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_D1_model.zip", sep=",")
joblib.dump(rand_frst_clf,PATH_DATA_Quant+"poc_dax_m30_rand_forest.pkl")

In [ ]:
# too long and too greedy ! 
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (20,20), dpi=800)
plot_tree(rand_frst_clf.estimators_[0], feature_names=X_cols.columns,
          class_names=y_cols.name, filled=True, impurity=True, rounded=True)
print("ok")
fig.savefig(PATH_DATA_Quant+"rf_tree_0.png")


In [ ]:
df_model['predict']=rand_frst_clf.predict(df_model[X_cols.columns])

In [ ]:
df_model[['m30_top_tp_touch','predict']].describe()

In [ ]:
df_dax_m30_d1=df_dax_m30_d1.join(df_model['predict'], how="left")
df_dax_m30_d1[:5]

In [ ]:
df_dax_m30_d1['predict']=df_dax_m30_d1['predict'].fillna(0)

In [ ]:
df_dax_m30_d1['predict'].describe()

In [ ]:
df_dax_m30_d1.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_D1_with_predict.zip", sep=",")

joblib.dump(rand_frst_clf,PATH_DATA_Quant+"poc_dax_m30_rand_forest.pkl")

###############################################################
 TEST OF REGRESSION 
##############################################################

In [ ]:
df_model_reg = df_dax_m30_d1[['m30_pos_hband_norm', 'm30_pos_lband_norm',   'm30_rsi14','m30_hour_norm', 'd1_pos_sma20_norm', 'd1_pos_sma50_norm',
                               'd1_pos_hband_norm', 'd1_pos_lband_norm', 'd1_vol_vol20_norm', 'd1_BBW_BBw20', 'd1_pos_sma200',  'd1_rsi14', 'd1_return', 'm30_perf_exit']]
df_model_reg.dropna(inplace=True)
df_model_reg.describe()


In [ ]:
X_cols_reg=df_model_reg[['m30_pos_hband_norm', 'm30_pos_lband_norm',   'm30_rsi14','m30_hour_norm', 'd1_pos_sma20_norm', 'd1_pos_sma50_norm',
                               'd1_pos_hband_norm', 'd1_pos_lband_norm', 'd1_vol_vol20_norm', 'd1_BBW_BBw20', 'd1_pos_sma200',  'd1_rsi14', 'd1_return']]
y_cols_reg=df_model_reg['m30_perf_exit']
X_cols_reg_valid=X_cols_reg['2020-07-01':]
y_cols_reg_valid=y_cols_reg['2020-07-01':]
X_cols_reg_train_test=X_cols_reg[:'2020-07-01']
y_cols_reg_train_test=y_cols_reg[:'2020-07-01']

In [ ]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_cols_reg_train_test, y_cols_reg_train_test, random_state=28, train_size=0.7)

In [ ]:
X_test_trim_reg = pd.concat([X_cols_reg_train_test['2015-04-01':'2015-06-30'],
                        X_cols_reg_train_test['2015-10-01':'2015-12-31'], X_cols_reg_train_test['2016-04-01':'2016-12-31'],X_cols_reg_train_test['2018-01-01':'2018-03-31'],X_cols_reg_train_test['2019-01-01':'2019-03-31']])
y_test_trim_reg = pd.concat([y_cols_reg_train_test['2015-04-01':'2015-06-30'],
                        y_cols_reg_train_test['2015-10-01':'2015-12-31'], y_cols_reg_train_test['2016-04-01':'2016-12-31'],y_cols_reg_train_test['2018-01-01':'2018-03-31'],y_cols_reg_train_test['2019-01-01':'2019-03-31']])
X_train_trim_reg=X_cols_reg_train_test.merge(X_test_trim_reg,indicator=True,how='left').loc[lambda x : x['_merge']!='both']
X_train_trim_reg.drop(axis=1, columns='_merge',inplace=True)
y_train_trim_reg=y_cols_reg_train_test[~y_cols_reg_train_test.index.isin(y_test_trim_reg.index)]
y_train_trim_reg.describe()


In [ ]:
# ,oob_score=True
rand_frst_clf_reg = RandomForestRegressor(n_estimators=180, bootstrap=True, criterion='squared_error',
                                          random_state=28, max_depth=6, min_samples_split=2, min_samples_leaf=5)
rand_frst_clf_reg.fit(X_train_reg, y_train_reg)


In [ ]:
rand_frst_clf_trim_reg = RandomForestRegressor(
    n_estimators=180, bootstrap=True, criterion='squared_error', random_state=28, max_depth=6, min_samples_split=2, min_samples_leaf=5)
rand_frst_clf_trim_reg.fit(X_train_trim_reg, y_train_trim_reg)


In [ ]:
y_pred_test_reg=rand_frst_clf_reg.predict(X_test_reg)
y_pred_test_trim_reg=rand_frst_clf_trim_reg.predict(X_test_trim_reg)
print(f"Accuracy shuffle :{rand_frst_clf_reg.score(X_test_reg,y_test_reg)*100}")
print(f"Accuracy quarter :{rand_frst_clf_trim_reg.score(X_test_trim_reg,y_test_trim_reg)*100}")

In [ ]:
y_pred_test_reg_valid=rand_frst_clf_reg.predict(X_cols_reg_valid)
y_pred_test_trim_reg_valid=rand_frst_clf_trim_reg.predict(X_cols_reg_valid)
print(f"Accuracy shuffle :{rand_frst_clf_reg.score(X_cols_reg_valid,y_cols_reg_valid)*100}")
print(f"Accuracy quarter :{rand_frst_clf_trim_reg.score(X_cols_reg_valid,y_cols_reg_valid)*100}")

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 50, stop =200, num = 8)] # 178
max_features = ['sqrt']
max_depth = [int(x) for x in np.linspace(6, 15, num = 4)] #6
min_samples_split = [2, 5, 10, 15, 100] # 2
min_samples_leaf = [1, 2, 5, 10] # 5

random_grid = {'n_estimators': n_estimators,
           'max_features': max_features,
           'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf}
           
# rf=RandomForestRegressor()

# rf_random = RandomizedSearchCV(estimator = rf,
#                 param_distributions = random_grid,
#                 #scoring='neg_mean_squared_error',
#                 n_iter = 100,
#                 cv = 3,
#                 verbose=1,
#                 random_state=28,
#                 n_jobs = 1)
# rf_random_trim = RandomizedSearchCV(estimator = rf,
#                 param_distributions = random_grid,
#                 #scoring='neg_mean_squared_error',
#                 n_iter = 100,
#                 cv = 3,
#                 verbose=1,
#                 random_state=28,
#                 n_jobs = 1)

# rf_random.fit(X_train_reg,y_train_reg)

# rf_random_trim.fit(X_train_trim_reg,y_train_trim_reg)


print(f"Accuracy train shuffle :{rf_random.score(X_train_reg,y_train_reg)*100}")
print(f"Accuracy train quart :{rf_random_trim.score(X_train_trim_reg,y_train_trim_reg)*100}")

print(f"Accuracy test shuffle :{rf_random.score(X_test_reg,y_test_reg)*100}")
print(f"Accuracy test quart :{rf_random_trim.score(X_test_trim_reg,y_test_trim_reg)*100}")

print(f"Accuracy qualif shuffle :{rf_random.score(X_cols_reg_valid,y_cols_reg_valid)*100}")
print(f"Accuracy qualif quart :{rf_random_trim.score(X_cols_reg_valid,y_cols_reg_valid)*100}")
    
report_best_scores(rf_random.cv_results_, 1)
report_best_scores(rf_random_trim.cv_results_, 1)

In [ ]:
feat_imp=rand_frst_clf_reg.feature_importances_
std_feat_imp = np.std([tree.feature_importances_ for tree in rand_frst_clf_reg.estimators_], axis=0)
pd.Series(feat_imp,index=X_cols_reg_valid.columns).sort_values(ascending=False)

In [ ]:
feat_imp=rand_frst_clf_trim_reg.feature_importances_
std_feat_imp = np.std([tree.feature_importances_ for tree in rand_frst_clf_trim_reg.estimators_], axis=0)
pd.Series(feat_imp,index=X_cols_reg_valid.columns).sort_values(ascending=False)

In [ ]:
df_model_reg['predict_reg_shuf']=rand_frst_clf_reg.predict(df_model_reg[X_cols_reg_valid.columns])
df_model_reg['predict_reg_trim']=rand_frst_clf_trim_reg.predict(df_model_reg[X_cols_reg_valid.columns])

In [ ]:
df_model_reg[['m30_perf_exit','predict_reg_shuf','predict_reg_trim']].describe()

In [ ]:
df_dax_m30_d1=df_dax_m30_d1.join(df_model_reg[['predict_reg_shuf','predict_reg_trim']], how="left")

In [ ]:
df_dax_m30_d1['predict_reg_shuf']=df_dax_m30_d1['predict_reg_shuf'].fillna(0)
df_dax_m30_d1['predict_reg_trim']=df_dax_m30_d1['predict_reg_trim'].fillna(0)
df_dax_m30_d1['meta_score_reg']=(df_dax_m30_d1['predict_reg_shuf']+df_dax_m30_d1['predict_reg_trim'])/2.0

In [ ]:
df_dax_m30_d1.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_D1_with_predict.zip", sep=",")
	
joblib.dump(rand_frst_clf_reg,PATH_DATA_Quant+"poc_dax_m30_rand_forest_reg_shuffle.pkl")
joblib.dump(rand_frst_clf_trim_reg,PATH_DATA_Quant+"poc_dax_m30_rand_forest_reg_trim.pkl")

In [ ]:
df_dax_pred = pd.read_csv(PATH_DATA_Quant+"DAX_M30_D1_with_predict.zip", sep=",",
                            index_col="Datetime", parse_dates=True, keep_default_na=True, low_memory=False)
df_dax_pred.sort_index(inplace = True)
df_dax_pred.info()

In [ ]:
df_dax_pred['meta_score_reg']=(df_dax_pred['predict_reg_shuf']+df_dax_pred['predict_reg_trim'])/2.0

In [ ]:
df_dax_m30_d1.dropna(inplace=True)
rmse_reg_shuf=metrics.mean_squared_error(df_dax_m30_d1['m30_perf_exit'], df_dax_m30_d1['predict_reg_shuf'])
rmse_reg_trim=metrics.mean_squared_error(df_dax_m30_d1['m30_perf_exit'], df_dax_m30_d1['predict_reg_trim'])
rmse_reg_meta=metrics.mean_squared_error(df_dax_m30_d1['m30_perf_exit'], df_dax_m30_d1['meta_score_reg'])
print(f" rmse shuffle : {rmse_reg_shuf} rmse trim : {rmse_reg_trim} rmse meta : {rmse_reg_meta}")

########################### 
START XGBOOST 
########################### 

In [ ]:
params = {
    'objective': 'reg:squarederror',
    'random_state': 28,
    'learning_rate': 0.03,
    'subsample': 0.8,
    'gamma': 0.15
}
xgb_reg_shuf = xgb.XGBRegressor(objective=params['objective'], random_state=params['random_state'],
                                learning_rate=params['learning_rate'], subsample=params['subsample'], gamma=params['gamma'])
xgb_reg_shuf.fit(X_train_reg, y_train_reg)
xgb_reg_trim = xgb.XGBRegressor(objective=params['objective'], random_state=params['random_state'],
                                learning_rate=params['learning_rate'], subsample=params['subsample'], gamma=params['gamma'])
xgb_reg_trim.fit(X_train_trim_reg, y_train_trim_reg)


In [ ]:
y_pred_test_reg = xgb_reg_shuf.predict(X_test_reg)
y_pred_test_trim_reg = xgb_reg_trim.predict(X_test_trim_reg)
print(
    f"Accuracy shuffle test :{xgb_reg_shuf.score(X_test_reg,y_test_reg)*100}")
print(
    f"Accuracy quarter test:{xgb_reg_trim.score(X_test_trim_reg,y_test_trim_reg)*100}")

y_pred_test_reg_valid = xgb_reg_shuf.predict(X_cols_reg_valid)
y_pred_test_trim_reg_valid = xgb_reg_trim.predict(X_cols_reg_valid)
print(
    f"Accuracy shuffle validation :{xgb_reg_shuf.score(X_cols_reg_valid,y_cols_reg_valid)*100}")
print(
    f"Accuracy quarter validation  :{xgb_reg_trim.score(X_cols_reg_valid,y_cols_reg_valid)*100}")


In [ ]:
xgb_reg_trim = xgb.XGBRegressor()

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_reg_trim, param_distributions=params, random_state=28, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train_trim_reg,y_train_trim_reg)

report_best_scores(search.cv_results_, 1)

In [ ]:
xgb.plot_importance(xgb_reg_trim)
plt.show()

In [ ]:
df_model_reg['predict_xgb_shuf']=xgb_reg_shuf.predict(df_model_reg[X_cols_reg_valid.columns])
df_model_reg['predict_xgb_trim']=xgb_reg_trim.predict(df_model_reg[X_cols_reg_valid.columns])

df_dax_m30_d1=df_dax_m30_d1.join(df_model_reg[['predict_xgb_shuf','predict_xgb_trim']], how="left")

df_dax_m30_d1['predict_xgb_shuf']=df_dax_m30_d1['predict_xgb_shuf'].fillna(0)
df_dax_m30_d1['predict_xgb_trim']=df_dax_m30_d1['predict_xgb_trim'].fillna(0)
df_dax_m30_d1['meta_score_xgb']=(df_dax_m30_d1['predict_xgb_shuf']+df_dax_m30_d1['predict_xgb_trim'])/2.0
df_dax_m30_d1['meta_score_rf_xgb']=(df_dax_m30_d1['meta_score_reg']+df_dax_m30_d1['meta_score_xgb'])/2.0

In [ ]:
df_dax_m30_d1.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_D1_with_predict.zip", sep=",")
	
joblib.dump(xgb_reg_shuf,PATH_DATA_Quant+"poc_dax_m30_xgb_reg_shuffle.pkl")
joblib.dump(xgb_reg_trim,PATH_DATA_Quant+"poc_dax_m30_xgb_reg_trim.pkl")